# Preprocessing

In [3]:
#pip install --upgrade scikit-learn

Could not fetch URL https://pypi.org/simple/pip/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/pip/ (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1129)'))) - skipping
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np # calculate the mean and standard deviation
import xgboost as xgb # XGBoost stuff
from sklearn.model_selection import train_test_split # split data into training and testing sets
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer # for scoring during
from sklearn.model_selection import GridSearchCV # cross validation
from sklearn.metrics import confusion_matrix # creates a confusion matrix
#from sklearn.metrics import plot_confusion_matrix # draws a confusion matrix

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [3]:
import pandas as pd # load and manipulate data and for One-Hot Encoding


In [4]:
df = pd.read_csv('data/rt_tidy.csv') # read in the csv file
df.head() # show the first five rows of the table

,Unnamed: 0,X.1,language,user_id,trait,stim_id,order,rt,rating,country,...,ethnicity.x,lab,block,X,Race,Gender,Age,ethnicity.y,gender,log_rt
0,1,606711,SPA,9654,confident,BM-040,41,1903,3,CO,...,NaN,COL_004,2,51,B,M,26.952381,black,male,7.551187
1,2,331376,HU,5381,responsible,WF-015,33,799,7,HU,...,"magyar, kaukázusi",HUN_001,2,93,W,F,24.211111,white,female,6.683361
2,3,162777,ENG,2734,attractive,LF-208,14,1226,1,US,...,Hispanic,USA_038,2,63,L,F,24.642857,latinx,female,7.111512
3,4,614094,SPA,9753,caring,BF-008,67,4561,7,CO,...,ninguno,COL_004,1,35,B,F,24.574713,black,female,8.425297
4,5,732055,PT,11726,aggressive,BF-201,36,912,3,PT,...,Caucasiana,POR_001,2,40,B,F,27.400000,black,female,6.815640


In [5]:
## change ethnicity.x into 

# create the new column that uses centralized rating
df['ratingcen'] = (df['rating'] - 5).abs() + 1
df.head()


,Unnamed: 0,X.1,language,user_id,trait,stim_id,order,rt,rating,country,...,lab,block,X,Race,Gender,Age,ethnicity.y,gender,log_rt,ratingcen
0,1,606711,SPA,9654,confident,BM-040,41,1903,3,CO,...,COL_004,2,51,B,M,26.952381,black,male,7.551187,3
1,2,331376,HU,5381,responsible,WF-015,33,799,7,HU,...,HUN_001,2,93,W,F,24.211111,white,female,6.683361,3
2,3,162777,ENG,2734,attractive,LF-208,14,1226,1,US,...,USA_038,2,63,L,F,24.642857,latinx,female,7.111512,5
3,4,614094,SPA,9753,caring,BF-008,67,4561,7,CO,...,COL_004,1,35,B,F,24.574713,black,female,8.425297,3
4,5,732055,PT,11726,aggressive,BF-201,36,912,3,PT,...,POR_001,2,40,B,F,27.400000,black,female,6.815640,3


In [6]:
df.drop(['Unnamed: 0', 'X.1', 'order', 'lab', 'block', 'ethnicity.y', 'gender', 'stim_id'], axis=1, inplace=True) # drop the first column


In [7]:
# Filter rows where ratingcen is 2 or 5
df = df[(df['ratingcen'] == 2) | (df['ratingcen'] == 5)]
df['ratingcen'] = df['ratingcen'].replace({2: 1, 5: 2})
df.head()

,language,user_id,trait,rt,rating,country,sex,age,ethnicity.x,X,Race,Gender,Age,log_rt,ratingcen
2,ENG,2734,attractive,1226,1,US,f,32.0,Hispanic,63,L,F,24.642857,7.111512,2
5,ENG,5692,attractive,1446,1,AU,m,24.0,Caucasian,39,B,F,34.213483,7.276556,2
6,ENG,6884,mean,1991,1,CA,f,19.0,South Asian,120,W,M,30.961538,7.596392,2
7,ENG,1541,old,1437,4,US,f,18.0,White,32,B,F,26.983696,7.270313,1
8,ENG,6897,caring,2546,6,CA,m,20.0,white,86,L,M,21.642857,7.842279,1


In [8]:
df.dtypes

language        object
user_id          int64
trait           object
rt               int64
rating           int64
country         object
sex             object
age            float64
ethnicity.x     object
X                int64
Race            object
Gender          object
Age            float64
log_rt         float64
ratingcen        int64
dtype: object

In [9]:
len(df.loc[df['ethnicity.x'] == ' '])
df['language'].unique()

array(['ENG', 'GER', 'RU', 'NOR', 'SPA', 'SLO', 'PT-BR', 'FR-BE', 'SRP',
       'ZH-S', 'TUR', 'SV', 'EL', 'HU', 'FRE', 'ZH-CN', 'FR-CH', 'RO',
       'NL', 'PT', 'THA', 'FAS', 'ES-PE', 'POL', 'ITA'], dtype=object)

In [10]:
df['ratingcen'] = df['ratingcen'].replace({1: 0, 2: 1})

# Split data into dependent and independent variables    

Split data into dependent and independent variables

In [11]:
y = df['ratingcen'].copy() # define the target variable (dependent variable) as y
X = df[['Age', 'Gender', 'Race', 'language', 'country', 'rt']].copy()



In [12]:
X_encoded = pd.get_dummies (X, columns= ['Race', 
                                         'Gender', 
                                         'language',
                                         'country']) # One Hot Encoding

X_encoded.head()

,Age,rt,Race_A,Race_B,Race_L,Race_W,Gender_F,Gender_M,language_EL,language_ENG,...,country_TH,country_TN,country_TR,country_TW,country_UA,country_UG,country_US,country_UY,country_VN,country_ZA
2,24.642857,1226,False,False,True,False,True,False,False,True,...,False,False,False,False,False,False,True,False,False,False
5,34.213483,1446,False,True,False,False,True,False,False,True,...,False,False,False,False,False,False,False,False,False,False
6,30.961538,1991,False,False,False,True,False,True,False,True,...,False,False,False,False,False,False,False,False,False,False
7,26.983696,1437,False,True,False,False,True,False,False,True,...,False,False,False,False,False,False,True,False,False,False
8,21.642857,2546,False,False,True,False,False,True,False,True,...,False,False,False,False,False,False,False,False,False,False


# Build xgboost model

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, random_state=42, stratify=y) # split the data set into training and testing sets

In [15]:
clf_xgb = xgb.XGBClassifier(objective='binary:logistic', missing = None, seed=42) # define the model
clf_xgb.fit(X_train, 
            y_train,
            verbose = True, 
            early_stopping_rounds=10, # use early stopping if it doesn't improve after 10 rounds
            #early_metric = 'aucpr',
            eval_set=[(X_test, y_test)]) # train the model


: 

: 

In [ ]:
optimal_params = GridSearchCV(
        estimator=xgb.XGBClassifier(objective='binary:logistic', seed=42, subsample=0.9, colsample_bytree=0.5),
        param_grid=param_grid,
        scoring = 'roc_auc',
        verbose = 0, # NOTE: If you want to see what Grid Search is doing, set verbose=2
        cv = 3
)

In [ ]:
optimal_params.fit(X_train, y_train, verbose=False, early_stopping_rounds=10, eval_metric='aucpr', eval_set=[(X_test, y_test)])


In [ ]:
clf_xgb = xgb.XGBClassifier(objective='binary:logistic',